<a href="https://colab.research.google.com/github/lbhargav/Convolutional-Neural-Networks/blob/master/CNN_Implementation_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation Tool Conv2D

In [0]:
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.contrib.slim.nets as nets

#Network_names
#Inception_V3, ResNet_50, Mobilenet_V1, VGG_19

# Internal memory size in bytes (16 MB)
internal_memory = 1024 * 1024 * 8  # 75 % of 8 MB, Internal memory
# External – internal memory bandwidth in bytes / second (10 GB / second)
bandwidth = 1024 * 1024 * 1024 * 10  # 32 bit DDR 3200 MHz, 50% available, 80% efficiency
# Matrix multiplication primitive size M, N, K in BLAS notation (32, 32, 32)
blas_m = 32
blas_n = 32
blas_k = 32
# Vector primitive size N x 1 (32)
blas_vec = 32
# Matrix primitive in parallel
# Number of vector primitives operating in parallel (1)
matrix_primitive_parallel = 1
# Matrix ops time
# Number of matrix multiplication primitive completions per second (1e9 / 32)
matrix_primitive_time = 1000000000 / 32
# Number of matrix multiplication primitives operating in parallel (1)
# Vector primitive in parallel
vector_primitive_parallel = 1
# Vector ops time
# Number of vector primitive completions per second (1e9)
vector_primitive_time = 1000000000
max_cycle = 32 * 32


In [0]:
# laoding model into graph
def load_graph(network_name):
    graph = tf.Graph()
    with graph.as_default():
        inputs = tf.placeholder(tf.float32,shape=[1,224,224,3],name='inputs')
        if network_name == "ResNet_50":
            nets.resnet_v1.resnet_v1_50(inputs=inputs)
        elif network_name == "Inception_V3":
            nets.inception.inception_v3(inputs = inputs)
        elif network_name == "VGG_19":
            nets.vgg.vgg_16(inputs = inputs)
        elif network_name == "Mobilenet_V1":
            mobilenet_v1_pb = 'D:\\workshop\\mobilenet_v1_1.0_224_frozen.pb'
            with tf.gfile.FastGFile(mobilenet_v1_pb, "rb") as f:
                graph_def = tf.GraphDef()
                graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name="")
        else:
            print("Unknown model...\n Expected ResNet_50 or Inception_V3 or VGG_19 or Mobilenet_V1")
    return(graph)  

In [0]:
#Expected ResNet_50 or Inception_V3 or VGG_19 or Mobilenet_V1
network_name = input()

Inception_V3


In [0]:
#loading the graph
graph = load_graph(network_name)

In [0]:
conv2D_df = pd.DataFrame(columns= ["name", "operation_name", "op_type", "Ni","Lr","Lc", "Dilations", "In Mem", "In Move", "No", "Mr", "Mc", "Out Mem", "Out Move",
                                   "G", "Fr", "Fc", "stride_r", "stride_c", "padding", "Cx Mem", "Cx Move",  "M", "N", "K", "MACs", "MACs Time", "filter_up_sampling"])

#getting operations
operations = graph.get_operations()

for operation in operations:
    layer_name = operation.node_def.name
    op_names.append(operation.type)

    if operation.op_def.name in("Conv2D", "MaxPool", "AvgPool"):

        op_type = "/".join([operation.type, str(operation.node_def.attr["strides"].list.i[1])])

        input0 = list(operation.inputs)[0].get_shape().as_list()
        output = operation.outputs[0].get_shape().as_list()

        ni = input0[3]
        lr = input0[1]
        lc = input0[2]
        no = output[3]
        mr = output[1]
        mc = output[2]

        dilation = operation.node_def.attr['dilations'].list.i
        
        if len(dilation)>0:
            filter_up_sampling = dilation[0]
        else:
            filter_up_sampling = 0

        in_mem = ni * lr * lc * 8
        if in_mem > internal_memory:
            in_mov = 1
        else:
            in_mov = 0

        out_mem = no * mr * mc * 8
        if out_mem > internal_memory:
            out_mov = 1
        else:
            out_mov = 0

        stride_r = operation.node_def.attr["strides"].list.i[1]
        stride_c = operation.node_def.attr["strides"].list.i[2]
        padding = operation.node_def.attr["padding"].s.decode("utf-8")

    if operation.op_def.name == "Conv2D":

        input1 = list(operation.inputs)[1].get_shape().as_list()
        fr = input1[0]
        fc = input1[1]
        g = input1[3]
        cx_mem = no * ni * fr * fc * 8
        cx_mov = 1
        m = no
        n = mr * mc
        k = (ni * fr * fc) + 1 
        mac = m * n * k/matrix_primitive_parallel
        mac_time = mac /(max_cycle * matrix_primitive_time)

        df = pd.DataFrame([layer_name, operation.op_def.name, op_type, ni, lr, lc, dilation, in_mem, in_mov, no, mr, mc, out_mem,
                                   out_mov, g, fr, fc, stride_r, stride_c, padding, cx_mem, cx_mov, m, n, k, mac,mac_time,filter_up_sampling]).T
        df.columns = conv2D_df.columns
        conv2D_df = conv2D_df.append(df, ignore_index= True)
    
    if (operation.op_def.name == "MaxPool" or operation.op_def.name== "AvgPool") :
        g = no
        fr = operation.node_def.attr['ksize'].list.i[1]
        fc = operation.node_def.attr['ksize'].list.i[2]
        
        cx_mem = 0
        cx_mov = 0

        m = ""
        n = ""
        k = ""

        mac = no * mr * mc / vector_primitive_parallel
        mac_time = mac / vector_primitive_time

        df = pd.DataFrame([layer_name, operation.op_def.name, op_type, ni, lr, lc, dilation, in_mem, in_mov, no, mr, mc, out_mem,
                out_mov, g, fr, fc, stride_r, stride_c, padding, cx_mem, cx_mov, m, n, k, mac,mac_time,filter_up_sampling]).T

        df.columns = conv2D_df.columns
        conv2D_df = conv2D_df.append(df, ignore_index= True)       

conv2D_df.head(5)

,name,operation_name,op_type,Ni,Lr,Lc,Dilations,In Mem,In Move,No,...,stride_c,padding,Cx Mem,Cx Move,M,N,K,MACs,MACs Time,filter_up_sampling
0,InceptionV3/InceptionV3/Conv2d_1a_3x3/Conv2D,Conv2D,Conv2D/2,3,224,224,"[1, 1, 1, 1]",1204224,0,32,...,2,VALID,6912,1,32,12321,28,1.10396e+07,0.000344988,1
1,InceptionV3/InceptionV3/Conv2d_2a_3x3/Conv2D,Conv2D,Conv2D/1,32,111,111,"[1, 1, 1, 1]",3154176,0,32,...,1,VALID,73728,1,32,11881,289,1.09875e+08,0.00343361,1
2,InceptionV3/InceptionV3/Conv2d_2b_3x3/Conv2D,Conv2D,Conv2D/1,32,109,109,"[1, 1, 1, 1]",3041536,0,64,...,1,SAME,147456,1,64,11881,289,2.19751e+08,0.00686722,1
3,InceptionV3/InceptionV3/MaxPool_3a_3x3/MaxPool,MaxPool,MaxPool/2,64,109,109,[],6083072,0,64,...,2,VALID,0,0,,,,186624,0.000186624,0
4,InceptionV3/InceptionV3/Conv2d_3b_1x1/Conv2D,Conv2D,Conv2D/1,64,54,54,"[1, 1, 1, 1]",1492992,0,80,...,1,VALID,40960,1,80,2916,65,1.51632e+07,0.00047385,1


In [0]:
#writing to csv
file_name = "./" + network_name + ".csv"
conv2D_df.loc[0, "In Move"] = 1
conv2D_df.loc[conv2D_df.shape[0] - 1, "Out Move"] = 1

conv2D_df["Band"] = conv2D_df.apply(func= lambda x: (x["In Mem"]*x["In Move"]) + (x["Out Mem"]*x["Out Move"]) + (x["Cx Mem"]*x["Cx Move"]), axis = 1)
conv2D_df["Time Band"] = conv2D_df["Band"]/bandwidth
conv2D_df["MACs Cycle"] = 32 * 32
conv2D_df["In Move Time"] = (conv2D_df["In Mem"] * conv2D_df["In Move"])/bandwidth
conv2D_df["Out Move Time"] = (conv2D_df["Out Mem"] * conv2D_df["Out Move"])/bandwidth
conv2D_df["Cx Move Time"] = (conv2D_df["Cx Mem"] * conv2D_df["Cx Move"])/bandwidth
conv2D_df["Input Up Sampling"] = "1"
conv2D_df["Output Down Sampling"] = conv2D_df["stride_r"]
conv2D_df["Total Data Move"] = conv2D_df.apply(func= lambda x: (x["In Mem"]*x["In Move"]) + (x["Out Mem"]*x["Out Move"]) + (x["Cx Mem"]*x["Cx Move"]), axis = 1)
conv2D_df["Total Data Time"] = conv2D_df["Total Data Move"]/bandwidth
conv2D_df["Serial Time"] = conv2D_df[["Total Data Time", "MACs Time"]].sum(axis=1)
conv2D_df["Parallel Time"] = conv2D_df[["Total Data Time", "MACs Time"]].max(axis=1)
conv2D_df = conv2D_df[['name', 'operation_name', 'op_type','Ni', 'Lr', 'Lc', 'In Mem', 'In Move', 'In Move Time',
                       'Input Up Sampling','No', 'Mr', 'Mc', 'Out Mem', 'Out Move', 'Out Move Time', 'Output Down Sampling','G', 'Fr', 'Fc', 'Cx Mem', 'Cx Move', 'Cx Move Time', 'stride_r', 'stride_c', 'padding','Band', 'Time Band', 
                        'M', 'N', 'K',  'MACs', 'MACs Time', 'Dilations', 'Serial Time', 'Parallel Time', "Input Up Sampling",
                        "Output Down Sampling"]]


# Append the total
total = pd.DataFrame(['Total', '', '',
       '', '', '', '', '', conv2D_df["In Move Time"].sum() , '',
       '', '', '', '', '', conv2D_df["Out Move Time"].sum(), '',
       '', '', '', '', '', conv2D_df["Cx Move Time"].sum(), '', '', '',
       '', conv2D_df["Time Band"].sum(), '', '', '', '', conv2D_df["MACs Time"].sum(), 
       '', conv2D_df["Serial Time"].sum(), conv2D_df["Parallel Time"].sum(), "", ""]).T
total.columns = conv2D_df.columns
conv2D_df = conv2D_df.append(total, ignore_index= True)

total_macs = pd.DataFrame(['', '', '',
       '', '', '', '', '', '', '',
       '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '',
       '', '', '', '', '',  "Total Mat compute", conv2D_df[conv2D_df["operation_name"].astype(str) == "Conv2D"]["MACs Time"].sum(), '', '', 
       '', '', ""]).T
total_macs.columns = conv2D_df.columns
conv2D_df = conv2D_df.append(total_macs, ignore_index= True) 

total_macs_pool = pd.DataFrame(['', '', '',
       '', '', '', '', '', '', '',
       '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '',
       '', '', '', '', '',  "Total vec compute", conv2D_df[(conv2D_df["operation_name"].astype(str) == "AvgPool") | (conv2D_df["operation_name"].astype(str) == "MaxPool")]["MACs Time"].sum(), '', '', 
       '', '', '']).T
total_macs_pool.columns = conv2D_df.columns

conv2D_df = conv2D_df.append(total_macs_pool, ignore_index= True)

conv2D_df.to_csv(file_name, header= True, index= False)